In [ ]:
# 🎮 EXPERIMENTA AQUÍ - Modifica estos valores:

MI_THRESHOLD = 0.002    # Prueba valores entre 0.0005 y 0.01
MI_N_SCALES = 5         # Prueba valores entre 3 y 6
MI_UPRIGHT = True       # True = más rápido, False = invariante a rotación

# Ejecutar con tus parámetros
print(f"Probando con tus parámetros:")
print(f"  threshold={MI_THRESHOLD}, n_scales={MI_N_SCALES}, upright={MI_UPRIGHT}")
print()

mi_surf = Surf(
    hessian_thresh=MI_THRESHOLD,
    n_scales=MI_N_SCALES,
    upright=MI_UPRIGHT
)

print("Detectando... ⏳")
mi_kps, mi_desc = mi_surf.detect_and_describe(img)

print(f"\n✓ Resultado: {len(mi_kps)} keypoints detectados!")

# Visualizar
fig, ax = plt.subplots(figsize=(14, 10))
draw_keypoints(img, mi_kps, mi_surf.sizes, color='cyan', show_orientation=not MI_UPRIGHT, ax=ax)
ax.set_title(f'Tu Configuración: {len(mi_kps)} keypoints\nthresh={MI_THRESHOLD}, scales={MI_N_SCALES}, upright={MI_UPRIGHT}', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 🧪 Experimento: Tu Turno

Modifica los parámetros en la siguiente celda y observa cómo cambian los resultados:

## 🎯 Conclusiones y Recomendaciones

### ¿Cuántos keypoints son suficientes?

- **< 30 keypoints**: Puede ser insuficiente, reducir `hessian_thresh`
- **30-100 keypoints**: ✅ Bueno para matching robusto (como este ejemplo)
- **100-500 keypoints**: ✅ Excelente cobertura
- **> 500 keypoints**: Puede tener ruido, aumentar `hessian_thresh`

### Optimización de Rendimiento

Si el procesamiento es lento (~40 segundos):

1. **Use U-SURF** (`upright=True`): 2-3x más rápido
2. **Reduzca escalas**: `n_scales=3` en vez de 4
3. **Aumente threshold**: `hessian_thresh=0.01` para menos keypoints
4. **Procese imágenes más pequeñas**: Resize antes de procesar

### Configuraciones Recomendadas

```python
# Para velocidad (desarrollo/prototipado)
surf_rapido = Surf(hessian_thresh=0.004, n_scales=3, upright=True)

# Para calidad (producción)
surf_calidad = Surf(hessian_thresh=0.002, n_scales=5, upright=False)

# Balanceado (recomendado)
surf_balanceado = Surf(hessian_thresh=0.003, n_scales=4, upright=False)
```

### Próximos Pasos

- Prueba con tus propias imágenes
- Experimenta con diferentes parámetros
- Implementa matching entre dos imágenes diferentes
- Usa los descriptores para reconocimiento de objetos

In [ ]:
# Estadísticas de descriptores
print("📊 Estadísticas de Descriptores SURF\n")
print(f"Shape: {descriptors.shape}")
print(f"Número de descriptores: {descriptors.shape[0]}")
print(f"Dimensiones por descriptor: {descriptors.shape[1]}")
print(f"\nValores:")
print(f"  Media: {descriptors.mean():.6f}")
print(f"  Desviación estándar: {descriptors.std():.6f}")
print(f"  Mínimo: {descriptors.min():.6f}")
print(f"  Máximo: {descriptors.max():.6f}")

# Verificar normalización
norms = np.linalg.norm(descriptors, axis=1)
print(f"\nNormas de los descriptores:")
print(f"  Media: {norms.mean():.6f} (debería ser ~1.0)")
print(f"  Min: {norms.min():.6f}")
print(f"  Max: {norms.max():.6f}")

# Visualizar distribución de un descriptor
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(descriptors.flatten(), bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Valor')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores en Descriptores')
plt.grid(alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(norms, bins=30, alpha=0.7, edgecolor='black', color='orange')
plt.xlabel('Norma')
plt.ylabel('Frecuencia')
plt.title('Distribución de Normas (debería estar cerca de 1.0)')
plt.axvline(1.0, color='red', linestyle='--', linewidth=2, label='Norma ideal = 1.0')
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 📈 Paso 6: Análisis de Descriptores

Analicemos las propiedades de los descriptores SURF generados.

In [ ]:
# Matching de descriptores (imagen consigo misma para demostración)
print("Realizando matching de descriptores...")

matches = match_descriptors(descriptors, descriptors, ratio=0.8)

print(f"✓ Matches encontrados: {len(matches)}")
print(f"✓ Ratio de matching: {len(matches)}/{len(keypoints)} = {len(matches)/len(keypoints)*100:.1f}%")
print()

# Mostrar algunos matches
print("Primeros 10 matches (idx1, idx2, distancia):")
print(f"{'#':<4} {'Index 1':<10} {'Index 2':<10} {'Distancia':<12}")
print("-" * 40)
for i, (idx1, idx2, dist) in enumerate(matches[:10]):
    print(f"{i:<4} {idx1:<10} {idx2:<10} {dist:<12.4f}")

## 🔗 Paso 5: Matching de Descriptores

SURF no solo detecta keypoints, también puede hacer matching entre dos imágenes. 
Para este demo, haremos matching de la imagen consigo misma (debería tener 100% de matches).

In [ ]:
# Comparar diferentes configuraciones
configs = [
    {"name": "Conservador (actual)", "hessian_thresh": 0.004, "n_scales": 4},
    {"name": "Más Keypoints", "hessian_thresh": 0.002, "n_scales": 4},
    {"name": "Muchos Keypoints", "hessian_thresh": 0.001, "n_scales": 5},
]

results = []

for config in configs:
    print(f"Probando: {config['name']}...")
    surf_test = Surf(
        hessian_thresh=config["hessian_thresh"],
        n_scales=config["n_scales"],
        upright=True  # Más rápido para este demo
    )
    kps, desc = surf_test.detect_and_describe(img)
    results.append((config['name'], len(kps), kps, surf_test.sizes))
    print(f"  → {len(kps)} keypoints detectados\n")

# Visualizar comparación
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, (name, n_kps, kps, sizes) in zip(axes, results):
    draw_keypoints(img, kps, sizes, color='yellow', show_orientation=False, ax=ax)
    ax.set_title(f'{name}\n{n_kps} keypoints', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Resumen:")
for name, n_kps, _, _ in results:
    print(f"  {name}: {n_kps} keypoints")

## 🎛️ Paso 4: Experimentar con Diferentes Parámetros

Probemos diferentes configuraciones para ver cómo afectan la detección.

In [ ]:
# Analizar los primeros keypoints
print("Información de los primeros 5 keypoints:\n")
print(f"{'#':<4} {'X':<8} {'Y':<8} {'Escala':<8} {'Response':<12} {'Ángulo (rad)':<12}")
print("-" * 70)

for i, (x, y, s, val, angle) in enumerate(keypoints[:5]):
    print(f"{i:<4} {x:<8.1f} {y:<8.1f} {s:<8} {val:<12.6f} {angle:<12.3f}")

print(f"\n... y {len(keypoints) - 5} keypoints más")

In [ ]:
# Visualizar keypoints detectados
fig, ax = plt.subplots(figsize=(14, 10))
draw_keypoints(img, keypoints, surf.sizes, color='lime', show_orientation=True, ax=ax)
ax.set_title(f'SURF Keypoints Detectados: {len(keypoints)}', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Los círculos verdes indican los keypoints.")
print("Las líneas muestran la orientación dominante de cada keypoint.")

## 📊 Paso 3: Visualizar Keypoints

Visualicemos los keypoints detectados sobre la imagen original.

In [ ]:
# Crear detector SURF con parámetros balanceados
surf = Surf(
    hessian_thresh=0.004,  # Threshold para detección
    n_scales=4,            # 4 escalas
    base_filter=9,         # Tamaño de filtro base
    step=6,                # Incremento entre escalas
    upright=False          # False = invariante a rotación
)

print(f"✓ Detector SURF creado")
print(f"  - Tamaños de filtro: {surf.sizes}")
print(f"  - Threshold: {surf.hessian_thresh}")
print(f"  - Rotación invariante: {not surf.upright}")
print()

# Detectar keypoints y descriptores
print("Detectando keypoints... (puede tomar ~40 segundos)")
keypoints, descriptors = surf.detect_and_describe(img)

print(f"\n✓ Detección completada!")
print(f"  - Keypoints detectados: {len(keypoints)}")
print(f"  - Descriptores shape: {descriptors.shape}")
print(f"  - Cada descriptor tiene {descriptors.shape[1]} dimensiones")

## 🔍 Paso 2: Detectar Keypoints con SURF

Ahora creamos un detector SURF y detectamos keypoints en la imagen.

### Parámetros principales:
- **`hessian_thresh`**: Umbral para detección (más bajo = más keypoints)
- **`n_scales`**: Número de escalas (más = más features en diferentes tamaños)
- **`upright`**: Si es True, no calcula orientación (2-3x más rápido)

In [ ]:
# Cargar imagen de ejemplo (Torre de Pisa)
image_path = Path("../images/image.png")

if not image_path.exists():
    print(f"⚠️  No se encontró la imagen en {image_path}")
    print("   Asegúrate de tener una imagen en la carpeta images/")
else:
    # Cargar y convertir a escala de grises
    img_pil = Image.open(image_path).convert('L')
    img = np.array(img_pil, dtype=np.float32) / 255.0
    
    print(f"✓ Imagen cargada: {img.shape[0]}x{img.shape[1]} píxeles")
    print(f"✓ Rango de valores: [{img.min():.2f}, {img.max():.2f}]")
    
    # Visualizar imagen original
    plt.figure(figsize=(10, 6))
    plt.imshow(img, cmap='gray')
    plt.title('Imagen Original - Torre di Pisa')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

## 📁 Paso 1: Cargar Imagen

Primero, cargamos una imagen y la convertimos a escala de grises normalizada [0, 1].

In [ ]:
# Importar bibliotecas necesarias
import sys
sys.path.insert(0, '..')  # Para importar pysurf desde el directorio padre

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

# Importar pySURF
from pysurf import Surf
from pysurf.match import match_descriptors
from pysurf.viz import draw_keypoints, draw_matches

print("✓ Bibliotecas importadas correctamente")
print(f"✓ pySURF listo para usar")

# 🎯 Demo Interactivo de pySURF

Bienvenido al notebook interactivo de **pySURF** - una implementación handmade de SURF (Speeded-Up Robust Features) en Python.

## ¿Qué es SURF?

SURF es un algoritmo de detección de features robusto y rápido que identifica puntos de interés (keypoints) en imágenes y genera descriptores únicos para cada uno. Es útil para:

- 🔍 Reconocimiento de objetos
- 📸 Matching de imágenes
- 🗺️ Panoramas y stitching
- 🎥 Tracking de objetos

## En este notebook

1. Cargar y preparar imágenes
2. Detectar keypoints con SURF
3. Visualizar resultados
4. Experimentar con diferentes parámetros
5. Matching entre imágenes